In [1]:
#!pip install -r requirements.txt

# LCEL & Runnables

### Quick Notes
- Expose schematic information about their input, output and config through their `.input_schema` property, the `.output_schema` property and the `.config_schema` method
- LCEL is a declarative way to compose Runnables into chains - any chain constructed with LCEL will also always automatically have sync, async, batch, and streaming support

### Main Primitives
#### 1. RunnableSequence
- Invokes a series of Runnables sequentially
- output of the previous runnable will be input to the next runnable in the chain
- can use pipe operator to construct or by passing a list of RunnableSequence
- 
#### 2. Runnable Parallel
- Invokes runnables concurrently, providing the same input to each. Construct using dict literal within a sequence or by passing a dict to RunnableParallel

#### 3. RunnablePassThrough
- Used as a "passthrough" take it takes any input to the current component  and allows us to provide it in the component output via the
  "Question" key
#### 4. Runnable Lambda 
- RunnableLambda converts a python callable into a Runnable.- 
Wrapping a callable in a RunnableLambda makes the callable usable within either a sync or async context- 

RunnableLambda can be composed as any other Runnable and provides seamless integration with LangChain tracing.

### Additional Methods
- can be used to modify behavior such as retry policy, lifecycle listeners, make them configurable 

### Debugging
- You can use `set_debug` from `from langchain_core.globals import set_debug`. set_debug(True)
- You can pass existing or custom callbacks to any give chain too:

```from langchain_core.tracers import ConsoleCallbackHandler

chain.invoke(
    ...,
    config={'callbacks': [ConsoleCallbackHandler()]}
)```

### Passing Data through
- RunnablePassthrough allows to pass inputs unchanged or with the addition of extra keys. This typically is used in conjuction with RunnableParallel to assign data to a new key in the map.
- RunnablePassthrough() called on it’s own, will simply take the input and pass it through.

In [1]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

- The `passed` key was called with RunnablePassthrough() and so it passed on {'num': 1}.

- In the second line with the key `extra`, we used RunnablePastshrough.assign with a lambda that multiplies the passed integer value by 3. So, `extra` was set with {'num': 1, 'mult': 3} which is the original value with the `'mult'` key added.

- Finally, we set a third key called `modified` in the map which uses a labmda to set a single value adding 1 to the num, which resulted in modified key with the value of 2.

### Run custom functions

#### NOTE
All inputs to these functions need to be a **SINGLE** argument. If you have a function that accepts multiple arguments, you should write a wrapper that accepts a single input and unpacks it into multiple argument.

- We use `itemgetter` to extract the input arguments
- `"a"` becomes `3` because `itemgetter` gets the value of `"foo"` and that is passed through pipe operator to the RunnableLamba which passes the value to the `length_function` which returns length of `"bar"` which is `3`
- `"b"` is 9 because we pass the dictionary of `{"text1": "bar", "text2": "gah"}` to the RunnableLambda that passes it into the `multiple_length_function`
- the dictionary of `{"a":3, "b": 9}` is then passed as output to the prompt usuing the pipe operator (`|`)
- the output of the prompt again is passed to the model
  

## Binding runtime arguments
- we can use `Runnable.bind()` to pass arguments as constants so that we can have access to them even within a runnable sequence where the argument is not part of the output of preceding runnables in the sequence

In [2]:
from langchain_core.runnables import RunnableLambda

# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)
print(sequence.invoke(1)) 
print(sequence.batch([1, 2, 3]))
print(sequence.stream(2))
await sequence.ainvoke(2)

4
[4, 6, 8]
<generator object RunnableSequence.stream at 0x0000024106C772E0>


6

In [3]:
sequence = RunnableLambda(lambda x: x + 2) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5),
    "pow_2": RunnableLambda(lambda x: x**2)
}
print(sequence.map().invoke([1,2,3])) 
sequence.batch([1,2,3])

[{'mul_2': 6, 'mul_5': 15, 'pow_2': 9}, {'mul_2': 8, 'mul_5': 20, 'pow_2': 16}, {'mul_2': 10, 'mul_5': 25, 'pow_2': 25}]


[{'mul_2': 6, 'mul_5': 15, 'pow_2': 9},
 {'mul_2': 8, 'mul_5': 20, 'pow_2': 16},
 {'mul_2': 10, 'mul_5': 25, 'pow_2': 25}]

In [4]:
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatAnthropic(model_name="claude-3-haiku-20240307",api_key='your_api_key")

chain = prompt | model | StrOutputParser()

In [5]:
chain.invoke({'topic':'friend'})

'Here\'s a funny joke about friends:\n\nWhy can\'t a bicycle stand up on its own? It\'s two-tired!\n\nThis joke plays on the similar pronunciation of "too tired" and "two-tired" to create a humorous pun about a bicycle not being able to stand up. The punchline is a play on words that relies on the shared sound between "too" and "two" to create an unexpected and silly joke.\n\nJokes about friends often involve humorous situations or misunderstandings between buddies. The shared experience and camaraderie between friends is a common theme in comedy. I hope you enjoyed this lighthearted joke! Let me know if you\'d like to hear another one.'

In [6]:
analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()

In [7]:
chain.invoke({'topic':'friend'})

'Here\'s a short and silly joke about friends:\n\nWhy can\'t a bicycle stand up on its own? It\'s two-tired!\n\n(Get it? "Two-tired" sounds like "too tired", implying the bicycle is tired and can\'t stand up on its own, just like a friend who is too tired to support you!)\n\nHumor is very subjective, so not everyone may find this joke funny. But the idea is to play on words in a lighthearted way to poke fun at the relationship between friends. Hopefully you get a chuckle out of this simple friend-related joke!'

In [8]:
composed_chain_with_lambda = (
    chain
    | (lambda input: {"joke": input})
    | analysis_prompt
    | model
    | StrOutputParser()
)

In [9]:
composed_chain_with_lambda.invoke({'topic':'dogs'})

'That\'s a cute pun-based dog joke! I chuckled at the play on the "two left feet" idiom. Puns can be a bit hit or miss, but this one lands nicely in the silly and mildly amusing category. Dog jokes can be a lot of fun - I\'d be happy to hear another one if you have another good pun or silly observation about our canine friends. Comedic delivery is an art, but you pulled this one off well!'

In [10]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'modified': 2}

In [11]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda, RunnableAssign

def add_num(a):
    return a+2
def mul_num(c):
    return c*4

chain = RunnableParallel({'add': RunnableLambda(add_num) , 'mul': RunnableLambda(mul_num)})

print(chain.map().invoke([2,3]))
chain.batch([2,3])

[{'add': 4, 'mul': 8}, {'add': 5, 'mul': 12}]


[{'add': 4, 'mul': 8}, {'add': 5, 'mul': 12}]

In [12]:

from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda, RunnableAssign
from typing import Dict

def mul_ten(x: Dict[str, int]):
    return {"added": x["input"] * 10}

mapper = RunnableParallel(
    {"add_step": RunnableLambda(mul_ten),}
)

runnable_assign = RunnableAssign(mapper)

# Synchronous example
print(runnable_assign.invoke({"input": 5}))


# Asynchronous example
await runnable_assign.ainvoke({"input": 5})


{'input': 5, 'add_step': {'added': 50}}


{'input': 5, 'add_step': {'added': 50}}

## Runnable Message History
- Runnable that manages chat message history for another Runnable.

- A chat message history is a sequence of messages that represent a conversation.

- RunnableWithMessageHistory wraps another Runnable and manages the chat message history for it; it is responsible for reading and updating the chat message history.

- The formats supports for the inputs and outputs of the wrapped Runnable are described below.

- RunnableWithMessageHistory must always be called with a config that contains the appropriate parameters for the chat message history factory.

- By default the Runnable is expected to take a single configuration parameter called session_id which is a string. This parameter is used to create a new or look up an existing chat message history that matches the given session_id.

In [33]:
import boto3
from typing import Optional
from typing import List
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.runnables import ConfigurableFieldSpec


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]):
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

def get_session_history(
    user_id: str, conversation_id: str):
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = InMemoryHistory()
    return store[(user_id, conversation_id)]
    
def get_by_session_id(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime', 
    region_name='us-east-1'
)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])
model = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0",client=bedrock_runtime)

chain = prompt | model
store= {}

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,                
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

with_message_history.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}}
)




AIMessage(content='Cosine is a trigonometric function that describes the ratio of the adjacent side to the hypotenuse of a right-angled triangle.\n\nMore specifically, in a right-angled triangle:\n\n- The cosine of an angle is the ratio of the length of the adjacent side to the length of the hypotenuse.\n\nThe cosine function is represented by the abbreviation cos or cos(), and is one of the basic trigonometric functions studied in mathematics along with sine and tangent.\n\nSome key properties of the cosine function:\n\n- It oscillates between -1 and 1\n- The cosine of 0° is 1\n- The cosine of 90° is 0 \n- The cosine function is periodic with a period of 360°\n- It is commonly used in physics, engineering, navigation and many other fields involving periodic phenomena.\n\nSo in summary, cosine expresses the ratio of sides in a right triangle in relation to one of the acute angles. It is an important concept in trigonometry and analytical geometry.', additional_kwargs={'usage': {'prompt